In [1]:
!unzip -qq -n "/content/drive/MyDrive/Datasets/Image Classification/full_animals.zip" -d "/content/sample_data/data"

In [2]:
%cd "/content/drive/MyDrive/Image Classification/CLS - tensorflow 2"

/content/drive/MyDrive/Image Classification/CLS - tensorflow 2


In [3]:
import os
import shutil
import argparse
import tensorflow as tf
import copy
import importlib
from utils.config_processing import load_config
from models import VGG16, CLS, TrainModel, get_labels
from augmenter import *

Can't import SyncBatchNormalization, using BatchNormalization


In [4]:
model_config  = load_config('./configs/models/convolution-base/vgg.yaml')
engine_config = load_config('./configs/engine/train_supervised.yaml')

2024-12-13 07:04:39,201 - INFO - Load file ./configs/models/convolution-base/vgg.yaml successfully
INFO:YOLO:Load file ./configs/models/convolution-base/vgg.yaml successfully
2024-12-13 07:04:39,707 - INFO - Load file ./configs/engine/train_supervised.yaml successfully
INFO:YOLO:Load file ./configs/engine/train_supervised.yaml successfully


# Models

In [ ]:
def build_models(config):
    config = copy.deepcopy(config)
    mod = importlib.import_module(__name__)
    input_shape = config.pop("input_shape")
    weight_path = config.pop("weight_path")
    load_weight_type = config.pop("load_weight_type")
    classes = config.pop('classes')

    architecture_config = config['Architecture']
    architecture_name = architecture_config.pop("name")

    backbone_config = config['Backbone']
    backbone_config['input_shape'] = input_shape
    if classes:
        if isinstance(classes, str):
            classes, num_classes = get_labels(classes)
        else:
            num_classes = len(classes)
        backbone_config['classes'] = num_classes

    backbone_name = backbone_config.pop("name")
    backbone = getattr(mod, backbone_name)(**backbone_config)

    architecture_config['backbone'] = backbone
    architecture = getattr(mod, architecture_name)(**architecture_config)

    model = TrainModel(architecture, classes=classes, name=architecture_name)
    return model

In [ ]:
model = build_models(model_config['Model'])

In [ ]:
model

<TrainModel name=train_model, built=False>

# Losses

In [ ]:
from losses.binary_crossentropy import BinaryCrossentropy
from losses.categorical_crossentropy import CategoricalCrossentropy
from losses.sparse_categorical_crossentropy import SparseCategoricalCrossentropy


def build_losses(config):
    config = copy.deepcopy(config)
    mod = importlib.import_module(__name__)
    losses = []

    for cfg in config:
        name = str(list(cfg.keys())[0])
        value = list(cfg.values())[0]
        if value:
            coeff = value.pop("coeff")
        else:
            coeff = 1
            value = {}
        arch = getattr(mod, name)(**value)
        losses.append({'loss': arch, 'coeff': coeff})
    return losses

In [ ]:
losses = build_losses(engine_config['Losses'])

# Optim

In [ ]:
from tensorflow.keras.optimizers import *


def build_optimizer(config):
    config = copy.deepcopy(config)
    name = config.pop("name")
    mod = importlib.import_module(__name__)
    arch = getattr(mod, name)(**config)
    return arch

In [ ]:
optimizer = build_optimizer(engine_config['Optimizer'])

# Augmentor

In [6]:
def parse_augment_config(aug_list):
    parsed = []
    for item in aug_list:
        for transform_name, params in item.items():
            transform_instance = eval(transform_name)
            if transform_instance is None:
                raise ValueError(f"Unknown transform: {transform_name}")
            if params is None:
                # For transforms without parameters
                parsed.append(transform_instance())
            elif isinstance(params, list):
                # For transforms that take a list (e.g., RandomOrder)
                nested = parse_augment_config(params)
                parsed.append(transform_instance(nested))
            elif isinstance(params, dict):
                # For transforms with parameters
                parsed.append(transform_instance(**params))
            else:
                raise ValueError(f"Invalid parameters for transform {transform_name}")
    return parsed


def build_augmenter(config):
    config = copy.deepcopy(config)

    if config:
        augmenter = parse_augment_config(config)

    return augmenter

In [7]:
augmentor = build_augmenter(engine_config['Dataset']['data_augmentation']['train'])

In [8]:
engine_config['Dataset']['data_augmentation']['train']

[{'RandomResizedCrop': {'size': 224}},
 {'RandomOrder': [{'CenterCrop': {'size': 224}},
   {'RandomOrder': [{'RandomOrder': [{'Posterize': {'bits': 4}},
       {'Rotation': {'degrees': 30}},
       {'Rotation': {'degrees': 40}},
       {'Rotation': {'degrees': 50}},
       {'Rotation': {'degrees': 60}}]}]},
   {'CenterCrop': {'size': 224}}]}]